In [1]:
import requests
import pandas as pd
import numpy as np
import pickle
import yfinance as yf

In [2]:
# Load the data from each file
file_paths = ["index_history_new.pkl", "saved_indexes.pkl", "saved_symbols.pkl", "ticker_history_new.pkl", "event_clean.pkl"]


data_dict = {}

for path in file_paths:
    with open(path, "rb") as file:
        data = pickle.load(file, encoding='latin1')
        data_dict[path] = data

# Access the loaded data
index_history = data_dict["index_history_new.pkl"] 
saved_indices = data_dict["saved_indexes.pkl"]
saved_tickers = data_dict["saved_symbols.pkl"]
ticker_history = data_dict["ticker_history_new.pkl"]
event_history = data_dict["event_clean.pkl"]

dict_tickers = ticker_history.copy()
dict_index = index_history.copy()

1. index_history has the last 5 years of index data for top 20 indices (from May 2019 - May 2023)
2. saved_indices has the top 20 index tickers
3. saved_tickers has the top 100 NASDAQ tickers
4. ticker_history has the last 5 years stock data for top 100 NASDAQ tickers (from May 2019 - May 2023)
5. event_history has the earnings date info from Jan 2019 until December 2023 for the saved_tickers

We might have to include data for index_history and ticker_history from Jan 2019 to May 2019 since the earnings data starts from Jan 2019-Dec 2023


1. index_history : dict with keys as index tickers. Value is a dataframe with 5 years of history (Date,Open,High,Low,Close,Volume,Dividends,Stock Splits)
2. saved_indices : list of keys of index_history
3. saved_tickers : list of keys of ticker_history
4. ticker_history : dict with keys as NASDAQ100 stock tickers. Value is a dataframe with 5 years of history (Date,Open,High,Low,Close,Volume,Dividends,Stock Splits)
5. event_history : dict with keys as NASDAQ100 stock tickers. Value is a dataframe with 5 years of history (Date,EPS Estimate, Reported EPS, Surprise)

In [3]:
# big = yf.Tickers(saved_tickers)
# dict_tickers_new = {i:big.tickers[i].history(period = "66mo") for i in saved_tickers}
# major_indices = yf.Tickers(saved_indices)
# dict_indices_new = {i:major_indices.tickers[i].history(period = "66mo") for i in saved_indices}

In [4]:
# with open('ticker_history_new.pkl', 'wb') as f:
#     pickle.dump(dict_tickers_new, f)
# with open('index_history_new.pkl', 'wb') as f:
#     pickle.dump(dict_indices_new, f)

In [5]:
us_indices = ['^GSPC','^DJI','^IXIC','^NYA','^XAX','^RUT']
closing_time_from_midnight = pd.DateOffset(hours=16) # Closing time of NYSE(4pm). Closing time matters because we want
# to determine if the closing time is before or after event. # For pre-market earning events, 
# the 'before' data should not include the data from that day. 

closing_time_from_midnight_ind = {}
us_indices = ['^GSPC','^DJI','^IXIC','^NYA','^XAX','^RUT']
for ind in us_indices:
    closing_time_from_midnight_ind[ind] = pd.DateOffset(hours=16) # Closing time of NYSE
closing_time_from_midnight_ind['^VIX'] = pd.DateOffset(hours=15.25)
closing_time_from_midnight_ind['^BUK100P'] = pd.DateOffset(hours=17.5)
closing_time_from_midnight_ind['^FTSE'] = pd.DateOffset(hours=16.5)
closing_time_from_midnight_ind['^GDAXI'] = pd.DateOffset(hours=18)
closing_time_from_midnight_ind['^FCHI'] = pd.DateOffset(hours=17.5)
closing_time_from_midnight_ind['^STOXX50E'] = pd.DateOffset(hours=18)
closing_time_from_midnight_ind['^N100'] = pd.DateOffset(hours=17.5)
closing_time_from_midnight_ind['^BFX'] = pd.DateOffset(hours=17.5)
closing_time_from_midnight_ind['IMOEX.ME'] = pd.DateOffset(hours=16)
closing_time_from_midnight_ind['^N225'] = pd.DateOffset(hours=15)
closing_time_from_midnight_ind['^HSI'] = pd.DateOffset(hours=16)
closing_time_from_midnight_ind['000001.SS'] = pd.DateOffset(hours=15)
closing_time_from_midnight_ind['399001.SZ'] = pd.DateOffset(hours=15)
closing_time_from_midnight_ind['^STI'] = pd.DateOffset(hours=17)

In [42]:
def earnings_data_no_of_calendar_days(num_days_before,num_days_after):

    d = {}
    indices = saved_indices

    for sym in {'ADBE':event_history['ADBE']}:#event_history:
        dum = []
        event_date = event_history[sym].index
        month_before_event_date = event_date - pd.DateOffset(days=num_days_before)
        week_after_event_date = event_date + pd.DateOffset(days=num_days_after)

        for j in [0]:#range(len(event_history[sym])):
            dic = {"Before":{}, "After":{}, "Earning":0}
            dic['Earning'] = event_history[sym].iloc[[j]]

            ticker_closing = ticker_history[sym].index+closing_time_from_midnight
            dfb = ticker_history[sym][["Close"]][(ticker_closing>month_before_event_date[j]) & (ticker_closing<=event_date[j])]
            dfb["Closing DateTime"] = dfb.index+closing_time_from_midnight
            dfb = dfb[["Closing DateTime","Close"]]
            dfb = dfb.rename(columns = {"Close":sym})
            dfb.sort_index(inplace=True)
            dfb.index = range(-len(dfb),0,1)
            dic["Before"][sym] = dfb
            for ind in indices:
                index_closing = index_history[ind].index+closing_time_from_midnight_ind[ind]
                dfbi = index_history[ind][["Close"]][(index_closing>month_before_event_date[j]) & (index_closing<=event_date[j])]            
                dfbi["Closing DateTime"] = dfbi.index+closing_time_from_midnight_ind[ind]
                dfbi = dfbi[["Closing DateTime","Close"]]
                dfbi = dfbi.rename(columns={"Close":ind})
                dfbi.sort_index(inplace=True)
                dfbi.index = range(-len(dfbi),0,1)
                dic["Before"][ind] = dfbi

            ticker_closing = ticker_history[sym].index+closing_time_from_midnight
            dfa = ticker_history[sym][["Close"]][(ticker_closing>event_date[j]) & (ticker_closing<=week_after_event_date[j])]
            dfa["Closing DateTime"] = dfa.index+closing_time_from_midnight
            dfa = dfa[["Closing DateTime","Close"]]
            dfa = dfa.rename(columns = {"Close":sym})
            dfa.sort_index(inplace=True)
            dfa.index = range(1,len(dfa)+1)
            dic["After"][sym] = dfa
            for ind in indices:
                index_closing = index_history[ind].index+closing_time_from_midnight_ind[ind]
                dfai = index_history[ind][["Close"]][(index_closing>event_date[j]) & (index_closing<=week_after_event_date[j])]            
                dfai["Closing DateTime"] = dfai.index+closing_time_from_midnight_ind[ind]
                dfai = dfai[["Closing DateTime","Close"]]
                dfai = dfai.rename(columns={"Close":ind})
                dfai.sort_index(inplace=True)
                dfai.index = range(1,len(dfai)+1)
                dic["After"][ind] = dfai
            dum.append(dic)
        d[sym] = dum
    return d

In [45]:
d = earnings_data_no_of_calendar_days(30,7)

In [46]:
d

{'ADBE': [{'Before': {'ADBE':              Closing DateTime        ADBE
    -21 2023-11-14 16:00:00-05:00  604.330017
    -20 2023-11-15 16:00:00-05:00  595.309998
    -19 2023-11-16 16:00:00-05:00  602.059998
    -18 2023-11-17 16:00:00-05:00  602.659973
    -17 2023-11-20 16:00:00-05:00  612.700012
    -16 2023-11-21 16:00:00-05:00  610.989990
    -15 2023-11-22 16:00:00-05:00  619.719971
    -14 2023-11-24 16:00:00-05:00  619.429993
    -13 2023-11-27 16:00:00-05:00  619.270020
    -12 2023-11-28 16:00:00-05:00  623.320007
    -11 2023-11-29 16:00:00-05:00  617.390015
    -10 2023-11-30 16:00:00-05:00  611.010010
    -9  2023-12-01 16:00:00-05:00  612.469971
    -8  2023-12-04 16:00:00-05:00  604.559998
    -7  2023-12-05 16:00:00-05:00  602.219971
    -6  2023-12-06 16:00:00-05:00  595.700012
    -5  2023-12-07 16:00:00-05:00  608.780029
    -4  2023-12-08 16:00:00-05:00  610.010010
    -3  2023-12-11 16:00:00-05:00  625.200012
    -2  2023-12-12 16:00:00-05:00  633.659973
    -1  

In [48]:
d1 = d['ADBE'][0]['Before']
print(d1)

{'ADBE':              Closing DateTime        ADBE
-21 2023-11-14 16:00:00-05:00  604.330017
-20 2023-11-15 16:00:00-05:00  595.309998
-19 2023-11-16 16:00:00-05:00  602.059998
-18 2023-11-17 16:00:00-05:00  602.659973
-17 2023-11-20 16:00:00-05:00  612.700012
-16 2023-11-21 16:00:00-05:00  610.989990
-15 2023-11-22 16:00:00-05:00  619.719971
-14 2023-11-24 16:00:00-05:00  619.429993
-13 2023-11-27 16:00:00-05:00  619.270020
-12 2023-11-28 16:00:00-05:00  623.320007
-11 2023-11-29 16:00:00-05:00  617.390015
-10 2023-11-30 16:00:00-05:00  611.010010
-9  2023-12-01 16:00:00-05:00  612.469971
-8  2023-12-04 16:00:00-05:00  604.559998
-7  2023-12-05 16:00:00-05:00  602.219971
-6  2023-12-06 16:00:00-05:00  595.700012
-5  2023-12-07 16:00:00-05:00  608.780029
-4  2023-12-08 16:00:00-05:00  610.010010
-3  2023-12-11 16:00:00-05:00  625.200012
-2  2023-12-12 16:00:00-05:00  633.659973
-1  2023-12-13 16:00:00-05:00  624.260010, '^GSPC':              Closing DateTime        ^GSPC
-21 2023-11-14

Use the above function if you want the data of number of certain calendar days prior/after the event. 

For example, d1 below would give all the data of the 20 indices and ticker history for 1 month prior to the event along with the closing datetime for each of those indices
d1 = d['ADBE'][0]['Before']



In [36]:
# Using last num_days_before trading days. We dont care about when those days occured relative to each other
def earnings_data_no_of_trading_days(num_days_before,num_days_after):

    d = {}
    
    indices = saved_indices

    for sym in event_history:#{'ADBE':event_history['ADBE']}:
        dum = []
        event_date = event_history[sym].index

        for j in range(len(event_history[sym])):
            dic = {"Before":0, "After":0, "Earning":0}
            dic['Earning'] = event_history[sym].iloc[[j]]

            ticker_closing = ticker_history[sym].index+closing_time_from_midnight
            dfb = ticker_history[sym][["Close"]][ticker_closing<=event_date[j]]
            dfb.sort_index(inplace=True)
            dfb = dfb[-num_days_before:]
            dfb = dfb.rename(columns = {"Close":sym})
            dfb.index = range(-len(dfb),0,1)
            for ind in indices:
                index_closing = index_history[ind].index+closing_time_from_midnight_ind[ind]
                dfbi = index_history[ind][["Close"]][index_closing<=event_date[j]]            
                dfbi.sort_index(inplace=True)
                dfbi = dfbi[-num_days_before:]
                dfbi = dfbi.rename(columns={"Close":ind})
                dfbi.index = range(-len(dfbi),0,1)
                dfb = dfb.join(dfbi)
            dic["Before"] = dfb

            ticker_closing = ticker_history[sym].index+closing_time_from_midnight
            dfa = ticker_history[sym][["Close"]][ticker_closing>event_date[j]]
            dfa.sort_index(inplace=True)
            dfa = dfa[:num_days_after]
            dfa = dfa.rename(columns = {"Close":sym})
            dfa.index = range(1,len(dfa)+1)
            for ind in indices:
                index_closing = index_history[ind].index+closing_time_from_midnight_ind[ind]
                dfai = index_history[ind][["Close"]][index_closing>event_date[j]]            
                dfai.sort_index(inplace=True)
                dfai = dfai[:num_days_after]
                dfai = dfai.rename(columns={"Close":ind})
                dfai.index = range(1,len(dfai)+1)
                dfa = dfa.join(dfai)
            dic["Before"] = dfa

            dum.append(dic)
        d[sym] = dum
    return d

Done


Use the above function if you only care about the data of number of trading days prior/after the event without being particular about closing times

In [22]:
d1 = d['ADBE'][0]['Before']
print(d1)

           ADBE        ^GSPC          ^DJI         ^IXIC          ^NYA  \
-30  544.500000  4237.859863  33274.578125  13061.469727  15002.549805   
-29  558.710022  4317.779785  33839.078125  13294.190430  15332.599609   
-28  563.659973  4358.339844  34061.320312  13478.280273  15475.200195   
-27  565.450012  4365.979980  34095.859375  13518.780273  15440.500000   
-26  585.200012  4378.379883  34152.601562  13639.860352  15388.490234   
-25  585.309998  4382.779785  34112.269531  13650.410156  15352.190430   
-24  577.739990  4347.350098  33891.941406  13521.450195  15235.179688   
-23  597.219971  4415.240234  34283.101562  13798.110352  15388.440430   
-22  590.340027  4411.549805  34337.871094  13767.740234  15403.139648   
-21  604.330017  4495.700195  34827.699219  14094.379883  15732.669922   
-20  595.309998  4502.879883  34991.210938  14103.839844  15765.019531   
-19  602.059998  4508.240234  34945.468750  14113.669922  15716.549805   
-18  602.659973  4514.020020  34947.28